# Python Wrappers

Python wrappers are functions that are added to another function which then can add additional functionality or modifies its behavior without directly changing its source code. They are typically implemented as decorators, which are special functions that take another **function** as **input** and apply some changes to its functionality.

Wrapper functions can be useful in various scenarios:

- Functionality Extension: We can add features like logging, performance measurement, or caching by wrapping our functions with a decorator.
- Code Reusability: We can apply a wrapper function or even a class to multiple entities, you can avoid code duplication, and ensure consistent behavior across different components.
- Behavior Modification: We can intercept the input arguments, for example, validate the input variable without the need for many assert lines.

# 1. Timer

This wrapper function measures the execution time of a function and prints the elapsed time. It can be useful for profiling and optimizing code.

In [1]:
import time

In [2]:
def timer(func):
    def wrapper(*args, **kwargs):
        # start the timer
        start_time = time.time()
        # call the decorated function
        result = func(*args, **kwargs)
        # remeasure the time
        end_time = time.time()
        # compute the elapsed time and print it
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time} seconds")
        # return the result of the decorated function execution
        return result
    # return reference to the wrapper function
    return wrapper

In [3]:
@timer
def train_model():
    print("Starting the model training function...")
    # simulate a function execution by pausing the program for 5 seconds
    time.sleep(5) 
    print("Model training completed!")

In [4]:
train_model()

Starting the model training function...
Model training completed!
Execution time: 5.012046575546265 seconds


# 2. Debugger

An additional useful wrapper function can be created to facilitate debugging by printing the inputs and outputs of each function. This approach allows us to gain insight into the execution flow of various functions without cluttering our applications with multiple print statements.

In [5]:
def debug(func):
    def wrapper(*args, **kwargs):
        # print the fucntion name and arguments
        print(f"Calling {func.__name__} with args: {args} kwargs: {kwargs}")
        # call the function
        result = func(*args, **kwargs)
        # print the results
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

We can use the `__name__` parameters to get the name of the function being called and then `args` , `kwargs` parameters to print what was passed to the function.

In [6]:
@debug
def add_numbers(x, y):
    return x + y

In [7]:
add_numbers(7, y=5,)

Calling add_numbers with args: (7,) kwargs: {'y': 5}
add_numbers returned: 12


12

# 3. Exception Handler

The `exception_handler` the wrapper will catch any exceptions raised within the `divide` function and handle them accordingly.

> We can customize the handling of exceptions within the wrapper function as per your requirements, such as logging the exception or performing additional error-handling steps.

In [8]:
def exception_handler(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            # Handle the exception
            print(f"An exception occurred: {str(e)}")
            # Optionally, perform additional error handling or logging
            # Reraise the exception if needed
    return wrapper

This becomes very useful to declutter our code and establish a unified procedure for handling exceptions and logging errors.

In [9]:
@exception_handler
def divide(x, y):
    result = x / y
    return result

In [10]:
divide(10, 0)

An exception occurred: division by zero


# 4. Input Validator

This wrapper function validates the input arguments of a function against specified conditions or data types. It can be used to ensure the correctness and consistency of the input data.

> The other approach to do that is by creating countless assert lines inside the function we want for validating the input data.

To add validations to the decoration, we need to wrap the decorator function with another function that takes in one or more validation functions as arguments. These validation functions are responsible for checking if the input values meet certain criteria or conditions.

The `validate_input` function itself acts as a decorator now. Inside the wrapper function, the input and the keyword arguments are checked against the provided validation functions. If any argument fails the validation, it raises a `ValueError` with a message indicating the invalid argument.

In [11]:
def validate_input(*validations):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i, val in enumerate(args):
                if i < len(validations):
                    if not validations[i](val):
                        raise ValueError(f"Invalid argument: {val}")
            for key, val in kwargs.items():
                if key in validations[len(args):]:
                    if not validations[len(args):][key](val):
                        raise ValueError(f"Invalid argument: {key}={val}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

To call the validated input, we need to define the validation functions. For example, two validation functions can be used. The first function (`lambda x: x > 0`) checks if the argument x is greater than 0, and the second function (`lambda y: isinstance(y, str)`) checks if the argument `y` is of a type string.

It is important to ensure that the order of the validation functions corresponds to the order of the arguments they are intended to validate.

In [12]:
@validate_input(lambda x: x > 0, lambda y: isinstance(y, str))
def divide_and_print(x, message):
    print(message)
    return 1 / x

In [13]:
divide_and_print(5, "Hello!")

Hello!


0.2

# 5. Retry

This wrapper retries the execution of a function a specified number of times with a delay between retries. It can be useful when dealing with network or API calls that may occasionally fail due to temporary issues.

To implement that we can define another wrapper function to our decorator, similar to our previous example. However this time rather than providing validation functions as input variables we can pass specific parameters such as the `max_attemps` and the `delay` .

When the decorated function is called, the `wrapper` function is invoked. It keeps track of the number of attempts made (starting at 0) and enters a while loop. The loop attempts to execute the decorated function and immediately returns the result if successful. However, if an exception occurs, it increments the attempts counter and prints an error message indicating the attempt number and the specific exception that occurred. It then waits for the specified delay using `time.sleep` before attempting the function again.

In [14]:
def retry(max_attempts, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"Attempt {attempts} failed: {e}")
                    time.sleep(delay)
            print(f"Function failed after {max_attempts} attempts")
        return wrapper
    return decorator

In order to invoke the function, we can specify the maximum number of attempts and the time duration in seconds between each invocation of the function.

In [15]:
@retry(max_attempts=3, delay=2)
def fetch_data(url):
    print("Fetching the data..")
    # raise timeout error to simulate a server not responding..
    raise TimeoutError("Server is not responding.")

In [16]:
fetch_data("https://example.com/data")

Fetching the data..
Attempt 1 failed: Server is not responding.
Fetching the data..
Attempt 2 failed: Server is not responding.
Fetching the data..
Attempt 3 failed: Server is not responding.
Function failed after 3 attempts
